# STEP 0: Persiapan Environment
Instalasi dan import pustaka yang diperlukan untuk preprocessing teks.

In [1]:
# Install library tambahan
!pip uninstall -y nltk Sastrawi
!pip install nltk Sastrawi swifter tqdm
!pip install sentence-transformers # Untuk S-BERT


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
Found existing installation: Sastrawi 1.0.1
Uninstalling Sastrawi-1.0.1:
  Successfully uninstalled Sastrawi-1.0.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached Sastrawi-1.0.1-py2.py3-none-any.whl.metadata (909 bytes)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached Sastrawi-1.0.1-py2.py3-none-any.whl (209 kB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\nurul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\nurul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Import pustaka
import pandas as pd
import re
import string
import nltk
import swifter
import os
import pickle
import numpy as np
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# --- SETTINGS UNTUK TAMPILAN PANDAS ---
pd.set_option('display.max_columns', None)  # Tampilkan semua kolom
pd.set_option('display.width', 1000)      # Atur lebar tampilan di konsol
pd.set_option('display.max_rows', None)   # Tampilkan semua baris (gunakan hati-hati untuk DataFrame besar)
# ------------------------------------

tqdm.pandas() # Untuk progress bar di swifter

# Download resource NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

swifter.set_defaults(display_progress=True)

C:\Users\nurul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# STEP 1: Load Dataset
Muat data lowongan kerja dari Dealls untuk diproses lebih lanjut.

In [3]:
# --- Konfigurasi File Input/Output ---
INPUT_CSV_FILE = 'dealls_jobs_tech.csv'
PREPROCESSED_CSV_FILE = 'preprocessed_jobs.csv'  # file hasil stemming

OUTPUT_MODEL_DIR = 'recommender_models'
os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)


In [4]:
# --- Load Data ---
if os.path.exists(PREPROCESSED_CSV_FILE):
    df = pd.read_csv(PREPROCESSED_CSV_FILE)
    print("✅ Data hasil preprocessing ditemukan. Lewati preprocessing dan lanjut ke TF-IDF.")
    skip_preprocessing = True
else:
    df = pd.read_csv(INPUT_CSV_FILE)
    print("📥 Menggunakan original data. Lanjut ke preprocessing.")
    skip_preprocessing = False

df.head()

✅ Data hasil preprocessing ditemukan. Lewati preprocessing dan lanjut ke TF-IDF.


,Kategori,Job Title,Company,Work Type,Work Mode,Experience,Salary,Detail Link,Job Description,Qualification,job_text,step_1_clean,step_2_lowercase,step_3_tokenisasi,step_4_stopword,step_5_stemmed,job_text_clean,Work Arrangement,Location
0,Data & Product,Product Manager [Bayarind],Sprint Asia,Penuh waktu,On-site • Jakarta Selatan,Min. 1 years of experience,Negotiable,https://dealls.com/loker/product-manager-bayar...,Collaborate with stakeholders to gather busine...,"Bachelor's degree in Computer Science, Busines...",Product Manager [Bayarind] Collaborate with st...,Product Manager Bayarind Collaborate with stak...,product manager bayarind collaborate with stak...,"['product', 'manager', 'bayarind', 'collaborat...","['product', 'manager', 'bayarind', 'collaborat...","['product', 'manager', 'bayarind', 'collaborat...",product manager bayarind collaborate stakehold...,On-site,Jakarta Selatan
1,Data & Product,Data Scientist Executive,detikcom,Penuh waktu,On-site • Jakarta,Min. 1 years of experience,Negotiable,https://dealls.com/loker/data-scientist-26~det...,"Design, develop, and implement machine learnin...",Bachelor’s Or Master’s Degree In Computer Scie...,"Data Scientist Executive Design, develop, and ...",Data Scientist Executive Design develop and im...,data scientist executive design develop and im...,"['data', 'scientist', 'executive', 'design', '...","['data', 'scientist', 'executive', 'design', '...","['data', 'scientist', 'executive', 'design', '...",data scientist executive design develop implem...,On-site,Jakarta
2,Data & Product,Billing Staff,Bayan Resources,Penuh waktu,On-site • Jakarta,Min. 1 years of experience,Negotiable,https://dealls.com/loker/billing-staff~bayan-r...,Check payment certificate from site to make su...,Bachelor’s degree in accounting\nMinimum 1 yea...,Billing Staff Check payment certificate from s...,Billing Staff Check payment certificate from s...,billing staff check payment certificate from s...,"['billing', 'staff', 'check', 'payment', 'cert...","['billing', 'staff', 'check', 'payment', 'cert...","['billing', 'staff', 'check', 'payment', 'cert...",billing staff check payment certificate site m...,On-site,Jakarta
3,Data & Product,Procurement Data Analyst Lead,Kenangan Brands (Kopi Kenangan),Penuh waktu,On-site • Jakarta,Min. 4 years of experience,Negotiable,https://dealls.com/loker/procurement-data-anal...,"Kopi Kenangan, a leading name in the F&B indus...",Proven experience in procurement analysis or a...,"Procurement Data Analyst Lead Kopi Kenangan, a...",Procurement Data Analyst Lead Kopi Kenangan a ...,procurement data analyst lead kopi kenangan a ...,"['procurement', 'data', 'analyst', 'lead', 'ko...","['procurement', 'data', 'analyst', 'lead', 'ko...","['procurement', 'data', 'analyst', 'lead', 'ko...",procurement data analyst lead kopi kenang lead...,On-site,Jakarta
4,Data & Product,Process Management System Analyst,"PT. Garudafood Putra Putri Jaya, Tbk",Penuh waktu,On-site • Jakarta,Min. 1 years of experience,Negotiable,https://dealls.com/loker/process-management-sy...,1. Memfasilitasi standardisasi proses\n2. Meny...,1. Mempunyai pemahaman terkait standardisasi p...,Process Management System Analyst 1. Memfasili...,Process Management System Analyst Memfasilitas...,process management system analyst memfasilitas...,"['process', 'management', 'system', 'analyst',...","['process', 'management', 'system', 'analyst',...","['process', 'management', 'system', 'analyst',...",process management system analyst fasilitas st...,On-site,Jakarta


In [5]:
df.isnull().sum()

Kategori              0
Job Title             0
Company               0
Work Type             0
Work Mode             0
Experience            0
Salary                0
Detail Link           0
Job Description       1
Qualification         0
job_text              0
step_1_clean          0
step_2_lowercase      0
step_3_tokenisasi     0
step_4_stopword       0
step_5_stemmed        0
job_text_clean        0
Work Arrangement      0
Location             37
dtype: int64

Dari preview data tersebut, akan dilakukan beberapa proses preprocesing, yaitu:

1.   Kolom *Job Title*, *Job Description*, dan kolom *Qualification* akan digabungkan menjadi satu kolom baru dengan nama *job_text* untuk memudahkan proses preprocessing teks.
2. Lalu, untuk kolom *Work Mode* akan dipecah menjadi kolom *mode* dan *location*.
3. Terakhir, akan dilakukan encoding terhadap kolom-kolom kategorikal

# STEP 2: Preprocessing
Pada proses ini, dilakukan preprocessing teks dan fitur kategorikal dari dataset lowongan pekerjaan dari Dealls. Output akhir adalah dataset bersih yang siap digunakan untuk model sistem rekomendasi.


## STEP 2.1 Preprocessing Content-Based (kolom job_text)
* Dilakukan penggabungan kolom Job Title, Job Description, dan Qualification menjadi satu kolom, yaitu *job_text*

* Dilakukan pembersihan data, seperti menghilangkan URL, simbol, angka, emoji, dan spasi berlebih dari job_text

* Dilakukan lowerisasi, tokenisasi, stopword removal, dan stemming

* Hasil stemming digabungkan kembali menjadi string lalu disimpan ke file CSV


In [6]:
# --- 2.1.2. Inisialisasi Stemmer --- 
factory_stemmer = StemmerFactory()
stemmer = factory_stemmer.create_stemmer()

factory_stopword = StopWordRemoverFactory()
stopword_remover = factory_stopword.create_stop_word_remover()

stop_words_en = set(stopwords.words('english'))
stop_words_id = set([
    'yang', 'dan', 'di', 'ke', 'dari', 'untuk', 'dengan', 'atau', 'pada', 'ini', 'itu',
    'sebagai', 'jika', 'ada', 'karena', 'saat', 'oleh', 'juga', 'agar', 'dalam', 'tidak',
    'adalah', 'bahwa', 'maupun', 'bagi', 'dapat', 'telah', 'sudah', 'lebih', 'harus',
    'setiap', 'kita', 'kami', 'saya', 'anda', 'mereka', 'semua', 'bisa', 'masih',
    'akan', 'dan', 'pun', 'hingga', 'dengan', 'berikut', 'namun', 'sehingga'
])
stop_words_all = stop_words_en.union(stop_words_id)

In [7]:
# --- 2.1.3. Definisi Fungsi Preprocessing ---
def tahap1_clean(text):
    if not isinstance(text, str): # Pastikan input adalah string
        return ""
    text = re.sub(r'http\S+|www\S+', '', text)                       # hapus URL
    text = re.sub(r'\d+', '', text)                                  # hapus angka
    text = text.translate(str.maketrans('', '', string.punctuation)) # hapus tanda baca & simbol
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)                       # hapus emoji dan karakter aneh
    text = re.sub(r'\s+', ' ', text).strip()                         # trim spasi berlebih
    return text

def tahap3_tokenisasi(text):
    return word_tokenize(text)

def tahap4_stopword(tokens):
    return [word for word in tokens if word not in stop_words_all and len(word) > 2] # Filter kata < 3 huruf

def tahap5_stemming_job(token_list):
    return [stemmer.stem(word) for word in token_list]

In [8]:
# --- 2.1.4. Fungsi gabungan untuk preprocessing teks baru (pipeline) --- 
def cleaning_pipeline(text):
    # Step 1: Bersihkan URL, simbol, angka, emoji
    text = tahap1_clean(text)

    # Step 2: Lowercase
    text = text.lower()

    # Step 3: Tokenisasi
    tokens = tahap3_tokenisasi(text)

    # Step 4: Stopword Removal
    tokens = tahap4_stopword(tokens)

    # Step 5: Stemming
    stemmed = tahap5_stemming_job(tokens)

    return ' '.join(stemmed)

In [9]:
# --- 2.1.5. Terapkan Pipeline ke DataFrame ---
if not skip_preprocessing:
    # Gabungkan kolom Job Title + Job Description + Qualification
    df['job_text'] = df["Job Title"].astype(str) + " " + df["Job Description"].astype(str) + " " + df["Qualification"].astype(str)

    print("\n🔄 Memulai pipeline preprocessing ke DataFrame...")

    df['step_1_clean'] = df['job_text'].swifter.apply(tahap1_clean)
    df['step_2_lowercase'] = df['step_1_clean'].str.lower()
    df['step_3_tokenisasi'] = df['step_2_lowercase'].swifter.apply(tahap3_tokenisasi)
    df['step_4_stopword'] = df['step_3_tokenisasi'].swifter.apply(tahap4_stopword)
    df['step_5_stemmed'] = df['step_4_stopword'].swifter.apply(tahap5_stemming_job)

    df['job_text_clean'] = df['step_5_stemmed'].apply(lambda x: ' '.join(x))

    df.to_csv(PREPROCESSED_CSV_FILE, index=False)
    print(f"✅ Preprocessing selesai, data disimpan ke {PREPROCESSED_CSV_FILE}")
else:
    print("📄 Kolom job_text_clean sudah tersedia, preprocessing dilewati.")


📄 Kolom job_text_clean sudah tersedia, preprocessing dilewati.


In [10]:
# --- 2.1.7. Simpan ke CSV supaya tidak perlu preprocess ulang selama data masih sama
df.to_csv('preprocessed_jobs.csv', index=False)
print("✅ STEP 2.1.7. - Preprocessing selesai, data disimpan ke preprocessed_jobs.csv")


✅ STEP 2.1.7. - Preprocessing selesai, data disimpan ke preprocessed_jobs.csv


In [25]:
# Contoh input
new_text = "Dicari kandidat yang memiliki pengalaman data engineering minimal 2 tahun dan siap kerja remote."

# Preprocess
cleaned_tokens = cleaning_pipeline(new_text)

print("Teks awal :", new_text)
print("\nTeks akhir:", cleaned_tokens)


Teks awal : Dicari kandidat yang memiliki pengalaman data engineering minimal 2 tahun dan siap kerja remote.

Teks akhir: cari kandidat milik alam data engineering minimal tahun siap kerja remote


## STEP 2.2: Pemecahan Kolom Work Mode dan Encoding
* DIlakukan pemecahan pada kolom 'Work Mode' menjadi dua kolom: 'Work Arrangement' dan 'Location'

In [12]:
# --- 2.2.1: Pecah kolom 'Work Mode' jadi dua kolom baru ---
df[['Work Arrangement', 'Location']] = df['Work Mode'].str.split(' • ', expand=True)

# Bersihkan spasi berlebih
df['Work Arrangement'] = df['Work Arrangement'].str.strip()
df['Location'] = df['Location'].str.strip()

print("✅ STEP 2.2.1 - Work Mode dipecah menjadi Work Arrangement dan Location:")
df[['Work Mode', 'Work Arrangement', 'Location']].head()

✅ STEP 2.2.1 - Work Mode dipecah menjadi Work Arrangement dan Location:


,Work Mode,Work Arrangement,Location
0,On-site • Jakarta Selatan,On-site,Jakarta Selatan
1,On-site • Jakarta,On-site,Jakarta
2,On-site • Jakarta,On-site,Jakarta
3,On-site • Jakarta,On-site,Jakarta
4,On-site • Jakarta,On-site,Jakarta


In [13]:
df.head()

,Kategori,Job Title,Company,Work Type,Work Mode,Experience,Salary,Detail Link,Job Description,Qualification,job_text,step_1_clean,step_2_lowercase,step_3_tokenisasi,step_4_stopword,step_5_stemmed,job_text_clean,Work Arrangement,Location
0,Data & Product,Product Manager [Bayarind],Sprint Asia,Penuh waktu,On-site • Jakarta Selatan,Min. 1 years of experience,Negotiable,https://dealls.com/loker/product-manager-bayar...,Collaborate with stakeholders to gather busine...,"Bachelor's degree in Computer Science, Busines...",Product Manager [Bayarind] Collaborate with st...,Product Manager Bayarind Collaborate with stak...,product manager bayarind collaborate with stak...,"['product', 'manager', 'bayarind', 'collaborat...","['product', 'manager', 'bayarind', 'collaborat...","['product', 'manager', 'bayarind', 'collaborat...",product manager bayarind collaborate stakehold...,On-site,Jakarta Selatan
1,Data & Product,Data Scientist Executive,detikcom,Penuh waktu,On-site • Jakarta,Min. 1 years of experience,Negotiable,https://dealls.com/loker/data-scientist-26~det...,"Design, develop, and implement machine learnin...",Bachelor’s Or Master’s Degree In Computer Scie...,"Data Scientist Executive Design, develop, and ...",Data Scientist Executive Design develop and im...,data scientist executive design develop and im...,"['data', 'scientist', 'executive', 'design', '...","['data', 'scientist', 'executive', 'design', '...","['data', 'scientist', 'executive', 'design', '...",data scientist executive design develop implem...,On-site,Jakarta
2,Data & Product,Billing Staff,Bayan Resources,Penuh waktu,On-site • Jakarta,Min. 1 years of experience,Negotiable,https://dealls.com/loker/billing-staff~bayan-r...,Check payment certificate from site to make su...,Bachelor’s degree in accounting\nMinimum 1 yea...,Billing Staff Check payment certificate from s...,Billing Staff Check payment certificate from s...,billing staff check payment certificate from s...,"['billing', 'staff', 'check', 'payment', 'cert...","['billing', 'staff', 'check', 'payment', 'cert...","['billing', 'staff', 'check', 'payment', 'cert...",billing staff check payment certificate site m...,On-site,Jakarta
3,Data & Product,Procurement Data Analyst Lead,Kenangan Brands (Kopi Kenangan),Penuh waktu,On-site • Jakarta,Min. 4 years of experience,Negotiable,https://dealls.com/loker/procurement-data-anal...,"Kopi Kenangan, a leading name in the F&B indus...",Proven experience in procurement analysis or a...,"Procurement Data Analyst Lead Kopi Kenangan, a...",Procurement Data Analyst Lead Kopi Kenangan a ...,procurement data analyst lead kopi kenangan a ...,"['procurement', 'data', 'analyst', 'lead', 'ko...","['procurement', 'data', 'analyst', 'lead', 'ko...","['procurement', 'data', 'analyst', 'lead', 'ko...",procurement data analyst lead kopi kenang lead...,On-site,Jakarta
4,Data & Product,Process Management System Analyst,"PT. Garudafood Putra Putri Jaya, Tbk",Penuh waktu,On-site • Jakarta,Min. 1 years of experience,Negotiable,https://dealls.com/loker/process-management-sy...,1. Memfasilitasi standardisasi proses\n2. Meny...,1. Mempunyai pemahaman terkait standardisasi p...,Process Management System Analyst 1. Memfasili...,Process Management System Analyst Memfasilitas...,process management system analyst memfasilitas...,"['process', 'management', 'system', 'analyst',...","['process', 'management', 'system', 'analyst',...","['process', 'management', 'system', 'analyst',...",process management system analyst fasilitas st...,On-site,Jakarta


In [14]:
# --- 2.2.2. Save hasil preprocessing ---
df.to_csv("final_preprocessed_jobs_tech.csv", index=False)
print("✅ STEP 2.2.2 - Dataset final disimpan ke final_preprocessed_jobs_tech.csv")

✅ STEP 2.2.2 - Dataset final disimpan ke final_preprocessed_jobs_tech.csv


# STEP 3: Text Vectorization

In [15]:
# --- 3.1 Inisialisasi TF-IDF Vectorizer ---

# max_features: Batas jumlah kata unik yang digunakan sebagai fitur.
#               Membantu mengurangi dimensi dan fokus pada kata-kata paling penting.
tfidf_vectorizer = TfidfVectorizer(max_features=5000) 

# Fitur dan transform data teks yang sudah diproses
# tfidf_matrix akan menjadi representasi numerik dari setiap lowongan
tfidf_matrix = tfidf_vectorizer.fit_transform(df['job_text_clean'])

print("✅ STEP 3.1 - Text Vectorization dengan TF-IDF selesai. \n\nBentuk matriks TF-IDF:")
print(tfidf_matrix)

# Simpan TF-IDF Vectorizer
with open(os.path.join(OUTPUT_MODEL_DIR, 'tfidf_vectorizer.pkl'), 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
print(f"\nTF-IDF Vectorizer disimpan ke '{os.path.join(OUTPUT_MODEL_DIR, 'tfidf_vectorizer.pkl')}'")

# Simpan Matriks TF-IDF
with open(os.path.join(OUTPUT_MODEL_DIR, 'tfidf_matrix.pkl'), 'wb') as f:
    pickle.dump(tfidf_matrix, f)
print(f"\nTF-IDF Matrix disimpan ke '{os.path.join(OUTPUT_MODEL_DIR, 'tfidf_matrix.pkl')}'")

✅ STEP 3.1 - Text Vectorization dengan TF-IDF selesai. 

Bentuk matriks TF-IDF:
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 83856 stored elements and shape (1138, 5000)>
  Coords	Values
  (0, 3499)	0.28083127466762947
  (0, 2695)	0.06525035208501327
  (0, 460)	0.1340362757699707
  (0, 800)	0.058154624013021114
  (0, 4303)	0.14523241420394492
  (0, 1798)	0.10257009739800517
  (0, 605)	0.10406678544299022
  (0, 4810)	0.20360602551177415
  (0, 3787)	0.13774950335566596
  (0, 4695)	0.09442661635193031
  (0, 763)	0.09442661635193031
  (0, 1199)	0.09879565344327129
  (0, 1272)	0.07302889356925019
  (0, 4345)	0.10930088590815938
  (0, 4967)	0.0429700979449773
  (0, 773)	0.07545248579728818
  (0, 1418)	0.05922245798460051
  (0, 1186)	0.09647749454644704
  (0, 4529)	0.11487509793608773
  (0, 1144)	0.08941346710128349
  (0, 1608)	0.08688595682548791
  (0, 1426)	0.12307445086867605
  (0, 333)	0.08131709701798479
  (0, 1138)	0.12102918639148816
  (0, 3868)	0.13072835094774993
  :

In [ ]:
# --- 3.2 Inisialisasi Text Vectorization: S-BERT ---
print("\n--- Memulai Tahap Text Vectorization: S-BERT ---")
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 

sbert_embeddings = sbert_model.encode(df['job_text_clean'].tolist(), show_progress_bar=True)

print("\n✅ STEP 3.2 - S-BERT Vectorization selesai. Bentuk matriks embeddings:", sbert_embeddings.shape)

with open(os.path.join(OUTPUT_MODEL_DIR, 'sbert_embeddings.pkl'), 'wb') as f:
    pickle.dump(sbert_embeddings, f)
print(f"\nS-BERT Embeddings disimpan ke '{os.path.join(OUTPUT_MODEL_DIR, 'sbert_embeddings.pkl')}'")

sbert_model.save_pretrained(os.path.join(OUTPUT_MODEL_DIR, 'sbert_model_dir'))
print(f"\nS-BERT Model (object) disimpan ke '{os.path.join(OUTPUT_MODEL_DIR, 'sbert_model_dir')}'")



--- Memulai Tahap Text Vectorization: S-BERT ---


Batches: 100%|██████████| 36/36 [01:02<00:00,  1.74s/it]


✅ STEP 3.2 - S-BERT Vectorization selesai. Bentuk matriks embeddings: (1138, 384)

S-BERT Embeddings disimpan ke 'recommender_models\sbert_embeddings.pkl'

S-BERT Model (object) disimpan ke 'recommender_models\sbert_model_dir'


# STEP 4: Similarity Mapping

In [27]:
# --- 4.1 Similarity Mapping tahap persiapan data ---
df_for_recommendation = df[['Job Title', 'Company', 'Job Description', 'Qualification', 'job_text_clean']]

with open(os.path.join(OUTPUT_MODEL_DIR, 'df_for_recommendation.pkl'), 'wb') as f:
    pickle.dump(df_for_recommendation, f)
print(f"DataFrame untuk rekomendasi disimpan ke '{os.path.join(OUTPUT_MODEL_DIR, 'df_for_recommendation.pkl')}'\n")
print("✅ STEP 4.1 - Similarity Mapping tahap persiapan data selesai.")


DataFrame untuk rekomendasi disimpan ke 'recommender_models\df_for_recommendation.pkl'

✅ STEP 4.1 - Similarity Mapping tahap persiapan data selesai.


In [28]:
# --- 4.2 Recommendation Modelling: Fungsi Umum untuk Kedua Model ---
def get_job_recommendations(course_description, model_type, num_recommendations=5):
    global tfidf_vectorizer, tfidf_matrix, sbert_model, sbert_embeddings, df_for_recommendation

    processed_course_text = cleaning_pipeline(course_description)
    
    if not processed_course_text:
        print("Peringatan: Deskripsi kursus kosong setelah preprocessing. Tidak dapat memberikan rekomendasi.")
        return pd.DataFrame()

    sim_scores = np.array([]) # Inisialisasi sim_scores

    if model_type == 'tfidf':
        if 'tfidf_vectorizer' not in globals() or 'tfidf_matrix' not in globals():
            print("Error: TF-IDF model belum dimuat. Pastikan Anda menjalankan skrip ini secara berurutan atau memuat model.")
            return pd.DataFrame()
        course_vector = tfidf_vectorizer.transform([processed_course_text])
        sim_scores = cosine_similarity(course_vector, tfidf_matrix).flatten()
    elif model_type == 'sbert':
        if 'sbert_model' not in globals() or 'sbert_embeddings' not in globals():
            print("Error: S-BERT model atau embeddings belum dimuat. Pastikan Anda menjalankan skrip ini secara berurutan atau memuat model.")
            return pd.DataFrame()
        course_vector = sbert_model.encode([processed_course_text])[0]
        sim_scores = cosine_similarity(course_vector.reshape(1, -1), sbert_embeddings).flatten()
    else:
        raise ValueError("model_type harus 'tfidf' atau 'sbert'.")
    
    if sim_scores.size == 0 or np.all(sim_scores == 0):
        print("Peringatan: Skor kemiripan semuanya nol atau kosong. Tidak ada rekomendasi yang kuat.")
        return pd.DataFrame()

    top_indices = sim_scores.argsort()[-num_recommendations:][::-1]

    # Filter out recommendations with 0 similarity score
    top_indices = [idx for idx in top_indices if sim_scores[idx] > 0]
    
    if not top_indices:
        return pd.DataFrame()

    recommended_jobs = df_for_recommendation.iloc[top_indices].copy()
    recommended_jobs['Similarity_Score'] = sim_scores[top_indices]

    return recommended_jobs[['Job Title', 'Company', 'Job Description', 'Qualification', 'Similarity_Score']]

print("✅ STEP 4.2 - Fungsi umum untuk kedua model berhasil dimuat")

✅ STEP 4.2 - Fungsi umum untuk kedua model berhasil dimuat


In [29]:
# --- 4.3 Contoh Penggunaan dan Perbandingan ---
print("✅ STEP 4.3 - Contoh Penggunaan & Perbandingan Model")
test_course_desc = "Saya baru saja menyelesaikan kursus mendalam tentang UI/UX Designer, saya memahami dengan baik metode design thinking"

print(f"\nQuery: {test_course_desc}")

# Rekomendasi menggunakan TF-IDF
print("\n--- Rekomendasi dengan TF-IDF ---")
recommended_jobs_tfidf = get_job_recommendations(
    test_course_desc,
    model_type='tfidf',
    num_recommendations=3
)
if not recommended_jobs_tfidf.empty:
    print(recommended_jobs_tfidf)
else:
    print("Tidak ada rekomendasi TF-IDF ditemukan.")

# Rekomendasi menggunakan S-BERT
print("\n\n\n--- Rekomendasi dengan S-BERT ---")
recommended_jobs_sbert = get_job_recommendations(
    test_course_desc,
    model_type='sbert',
    num_recommendations=3
)
if not recommended_jobs_sbert.empty:
    print(recommended_jobs_sbert)
else:
    print("Tidak ada rekomendasi S-BERT ditemukan.")

✅ STEP 4.3 - Contoh Penggunaan & Perbandingan Model

Query: Saya baru saja menyelesaikan kursus mendalam tentang UI/UX Designer, saya memahami dengan baik metode design thinking

--- Rekomendasi dengan TF-IDF ---
                    Job Title               Company                                    Job Description                                      Qualification  Similarity_Score
712  Quality Assurance Tester             ORDO Apps                                    Tanggung Jawab:  Pemahaman mendalam mengenai Quality Assurance ...          0.212084
304            UI/UX Designer  Pintar Ventura Group  We are building Klikoo, a digital financial so...  2–4 years of experience as a UI/UX Designer (s...          0.197519
155          Product Designer          Tentang Anak  Role Overview:\nA Product Designer will suppor...  4-6 years of experience as a UI/UX Designer, P...          0.192081



--- Rekomendasi dengan S-BERT ---
                     Job Title                   Company      

In [31]:
# --- Membuat Contoh Output JSON untuk Front-End ---
print("\n--- Membuat Contoh Output JSON untuk Front-End ---")

# Definisikan beberapa query contoh
example_queries = [
    "Saya baru saja menyelesaikan kursus mendalam tentang machine learning menggunakan Python, scikit-learn, dan TensorFlow. Saya juga mempelajari deep learning dan natural language processing.",
    "Saya memiliki sertifikasi AWS Solutions Architect dan ingin mencari pekerjaan terkait infrastruktur cloud.",
    "Saya belajar tentang manajemen produk, desain UX/UI, dan metodologi Agile."
]

all_example_recommendations = {}

for i, query in enumerate(example_queries):
    print(f"Menghasilkan rekomendasi untuk: {query}")
    # Gunakan model S-BERT karena Anda memutuskan untuk fokus padanya
    recs = get_job_recommendations(
        query,
        model_type='sbert', 
        num_recommendations=5 # Total jumlah rekomendasi per query
    )
    if not recs.empty:
        # Konversi DataFrame ke list of dictionaries, lalu ke string JSON
        all_example_recommendations[f"query_{i+1}"] = {
            "query_text": query,
            "recommendations": recs.to_dict(orient='records')
        }
    else:
        all_example_recommendations[f"query_{i+1}"] = {
            "query_text": query,
            "recommendations": []
        }

# Simpan semua contoh rekomendasi ke file JSON
OUTPUT_JSON_FILE_NAME = 'PP_MLOps_Nurul Ainil Fitri_Pricillia Silfany_Output.json'
with open(OUTPUT_JSON_FILE_NAME, 'w', encoding='utf-8') as f:
    json.dump(all_example_recommendations, f, indent=4, ensure_ascii=False)
print(f"\nContoh output JSON rekomendasi disimpan ke '{OUTPUT_JSON_FILE_NAME}'")

print("\nPelatihan model dan pembuatan output JSON selesai.")


--- Membuat Contoh Output JSON untuk Front-End ---
Menghasilkan rekomendasi untuk: Saya baru saja menyelesaikan kursus mendalam tentang machine learning menggunakan Python, scikit-learn, dan TensorFlow. Saya juga mempelajari deep learning dan natural language processing.
Menghasilkan rekomendasi untuk: Saya memiliki sertifikasi AWS Solutions Architect dan ingin mencari pekerjaan terkait infrastruktur cloud.
Menghasilkan rekomendasi untuk: Saya belajar tentang manajemen produk, desain UX/UI, dan metodologi Agile.

Contoh output JSON rekomendasi disimpan ke 'PP_MLOps_Nurul Ainil Fitri_Pricillia Silfany_Output.json'

Pelatihan model dan pembuatan output JSON selesai.
